In [1]:
import numpy as np

In [2]:
def init_random_params(layer_sizes, rs=np.random.RandomState(0)):
    params = []
    for m, n in zip(layer_sizes[:-1], layer_sizes[1:]):
        e = np.sqrt(6/(m + n))
        params.append([2 * e * rs.rand(n, m) - e,   # weight matrix
             2 * e * rs.rand(n, 1) - e]) # bias vector
    return np.array(params)

In [3]:
batch_size = 250
def get_data():
    t = np.linspace(-0.5,0.5,num=batch_size, endpoint=True)
    t.shape = (1, batch_size)
    return t

In [9]:
def sigmoid(x):
    return 1/(1 + np.exp(-x))

# Return the softplus of variable x
def softplus(x):
    return np.log(1 + np.exp(x))

def relu(x):
    return np.maximum(0, x)

# Choose activation
activation = softplus
activation_grad = sigmoid

def forward_prop(params, inputs):
    fp = []
    fp.append([inputs])
    for W, b in params:
        z = np.dot(W, inputs) + b
        inputs = activation(z)
        fp.append([z, inputs])
    del fp[-1][-1]
    return fp

In [5]:
def cost_function(y_hat, y):
    return np.sum((y - y_hat) ** 2)/ (2 * y_hat.shape[1])

def dE_dy_hat(y_hat, y):
    return (y_hat - y) / y_hat.shape[1]

In [17]:
def backward_prop(y, params, forward_prop):
    # delta (l) = dE_dz(l)
    # deltas also give the derivate with respect to the bias paramaters
    deltas = [0 for _ in range(len(params) + 1)]
    # print(deltas)
    
    grad_params = [0 for _ in range(len(params))]
    
    deltas[-1] = dE_dy_hat(forward_prop[-1][0], y)
    
    for i in range(-2, -1*(len(forward_prop)) - 1, -1):
        deltas[i] = np.multiply(np.dot(np.transpose(params[i+1][0]), deltas[i+1]), activation_grad(forward_prop[i][0]))
        grad_params[i+1] = [np.dot(deltas[i+1], np.transpose(forward_prop[i][-1])), np.reshape(np.sum(deltas[i+1], axis = 1),(deltas[i+1].shape[0],1))] 
    return np.array(grad_params)

In [12]:
'''
https://www.coursera.org/learn/machine-learning/resources/EcbzQ
epsilon = 1e-4;
for i = 1:n,
  thetaPlus = theta;
  thetaPlus(i) += epsilon;
  thetaMinus = theta;
  thetaMinus(i) -= epsilon;
  gradApprox(i) = (J(thetaPlus) - J(thetaMinus))/(2*epsilon)
end
'''
def gradient_checker(params, grad_params, y, inputs):
    epsilon = 10**(-4)
    for i in range(len(params)):
        for j in range(len(params[i])):
            if params[i][j].shape[1] > 1:
                for k in range(len(params[i][j])):
                    for l in range(len(params[i][j][k])):
                        params[i][j][k][l] += epsilon
                        y_hat = forward_prop(params, inputs)[-1][0]
                        J_theta_plus = cost_function(y_hat, y)
                        params[i][j][k][l] -= (2 * epsilon)
                        y_hat = forward_prop(params, inputs)[-1][0]
                        J_theta_minus = cost_function(y_hat, y)
                        params[i][j][k][l] += epsilon
                        grad_approx = (J_theta_plus - J_theta_minus) / (2 * epsilon)
                        print(abs(grad_params[i][j][k][l] - grad_approx) * 100 / abs(grad_params[i][j][k][l]))
            else:
                for k in range(len(params[i][j])):
                    params[i][j][k] += epsilon
                    y_hat = forward_prop(params, inputs)[-1][0]
                    J_theta_plus = cost_function(y_hat, y)
                    params[i][j][k] -= (2 * epsilon)
                    y_hat = forward_prop(params, inputs)[-1][0]
                    J_theta_minus = cost_function(y_hat, y)
                    params[i][j][k] += epsilon
                    grad_approx = (J_theta_plus - J_theta_minus) / (2 * epsilon)
                    print(abs(grad_params[i][j][k] - grad_approx) * 100 / abs(grad_params[i][j][k]))

In [38]:
'''
g_t^(2) indicates the elementwise
square g_t  g_t. Good default settings for the tested machine learning problems are α = 0.001,
β1 = 0.9, β2 = 0.999 and e = 10−8
. All operations on vectors are element-wise
Require: α: Stepsize
Require: β1, β2 ∈ [0, 1): Exponential decay rates for the moment estimates
Require: f(θ): Stochastic objective function with parameters θ
Require: θ0: Initial parameter vector
m0 ← 0 (Initialize 1st moment vector)
v0 ← 0 (Initialize 2nd moment vector)
t ← 0 (Initialize timestep)
while θt not converged do
t ← t + 1
gt ← ∇θft(θt−1) (Get gradients w.r.t. stochastic objective at timestep t)
mt ← β1 · mt−1 + (1 − β1) · gt (Update biased first moment estimate)
vt ← β2 · vt−1 + (1 − β2) · gt**2
(Update biased second raw moment estimate)
mbt ← mt/(1 − β1^t) (Compute bias-corrected first moment estimate)
vbt ← vt/(1 − β2^t)

(Compute bias-corrected second raw moment estimate)
θt ← θt−1 − α · mbt/(√vbt + e) (Update parameters)
end while
return θt (Resulting parameters)
'''
def adam_optimization(inputs, y, params, no_of_steps=1000, alpha = 0.001, beta1 = 0.9, beta2 = 0.999, epsilon = 10**(-8)):
    m = 0
    v = 0
    for i in range(1, no_of_steps+1):
        g = backward_prop(y, params, forward_prop(params, inputs))
        m = beta1 * m + (1- beta1)*g
        v = beta2 * v + (1- beta2)*(g**2)
        m_tilde = m/(1 - beta1**i)
        v_tilde = v/(1 - beta2**i)
        sub = alpha * m_tilde / (v_tilde + epsilon)**(0.5)
        params -= sub
        if i%100 == 0:
            print(cost_function(y, forward_prop(params, inputs)[-1][0]))
    return params

In [39]:
init_params = init_random_params([2, 4, 3, 2])
x = np.random.RandomState(0).randn(2, 10)
y = np.array([np.sin(x[0]), np.cos(x[1])])
optimized_params = adam_optimization(x, y, init_params, no_of_steps=8500)

3.36406008348
2.19626046916
1.58291809595
1.20989995796
0.961737887296
0.786818014374
0.658637891705
0.562133868535
0.488067943424
0.43043755421
0.385162954139
0.349369139226
0.320967792082
0.298401506665
0.280481144018
0.266279346716
0.255059326327
0.246226540281
0.239295575785
0.233867256156
0.229612594108
0.22626123029
0.223592685824
0.221429270289
0.219629891371
0.218084335145
0.216707831646
0.215435887579
0.2142194592
0.213020562974
0.211808395863
0.210555980507
0.209237282327
0.207824680666
0.206286624344
0.204585267823
0.202673869131
0.200493737402
0.197970555511
0.19500999743
0.191492769088
0.187269647716
0.18215799034
0.175942705361
0.168386105517
0.159247831811
0.148294609894
0.135216367293
0.119225094009
0.097881185731
0.0686785090559
0.0486242194945
0.0395930476521
0.0335064675969
0.0289075545563
0.0253666305552
0.0226234566032
0.0204722673257
0.0187450181117
0.0173149974525
0.0160941265327
0.0150222323983
0.0140552160994
0.0131562506718
0.0122901576631
0.0114197859062
0.01

In [40]:
forward_prop(inputs=x, params=optimized_params)[-1][0]

array([[ 0.88355816,  0.39934334,  0.83504008,  0.8914434 ,  0.94544585,
        -0.82423043,  0.80687031, -0.14928999, -0.11735898,  0.39314023],
       [ 0.99431747,  0.11404552,  0.63592369,  1.03702934,  0.92463925,
         1.00307311,  0.11018637,  0.87594322,  0.94010261,  0.705511  ]])

In [41]:
y

array([[ 0.9813841 ,  0.38956314,  0.82979374,  0.78376151,  0.95628847,
        -0.82897801,  0.81346693, -0.15077996, -0.10303566,  0.39915816],
       [ 0.98964365,  0.11625932,  0.72412071,  0.99260672,  0.90309941,
         0.94484532,  0.07664202,  0.97902875,  0.95139326,  0.65690057]])